# Project

In [1]:
student_name = 'Yutae Lee'

# Overview

In this project you will implement a both a serial version of mergesort as well as a parallel version of mergesort.  There are many variations of this algorithm and for this project we will use a simplified data parallel approach with the following assumptions:
1. The data is to be sorted from small to large.
1. The data may contain duplicates.
1. The data to be sorted fits easily in main memory.
1. The system used contains one or more multi-core CPU's.
1. The parallel version of merge sort will use the serial version of mergesort within each process.


## MergeSort

Merge sort is a divide and conquer sorting algorithm originally created by John von Neumann in 1945.  The algorithm basically divides the input list into $n$ sublists, where each sublist contains only a single element (and is thus sorted).  The algorithm then merges each of these sublists, typically pairwise, forming larger sorted sublists, and repeats this process until only one sorted list remains.  See [Wikipedia](https://en.wikipedia.org/wiki/Merge_sort).


## Algorithm Overview

In general the mergesort algorithm performs the following operations:

- Divide the input list into $n$ sublists, each containing a single element.  
- Repeat until only one list remains
    - Take sublists, pairwise, and merge them by greedily taking the smallest element from either sublist first.

The above algorithm can be broken up into two functions:
1. A recursive mergesort function that breaks its input into two equally sized lists and calls itself twice, passing half of the input data to each call.
1. A merge function that receives two sorted sublists as input and merges them into a single sorted list.

The following two subsections give a brief description of a possible implementation.  

**Note:** For the serial implementation you will need to write two functions, one for mergesort and another for merge as these functions will be reused in your parallel implementation.

### MergeSort

- Divide the input list into two sublists, roughly equal in length.
- Recursively call mergesort on each sublist, stopping the recursion when the input sublist contains only a single element.
- Merge the two sorted sublists returned by the recursion.

### Merge

- Given two input lists (both of which are assumed to be sorted).
- Create *pointers* (e.g. indices) that keep track of the current element in each list.  Initialize both pointers to the first element of each list.
- Compare the values of the current element in both lists.  
- Copy the smaller of the two to an output list.
- Move the pointer corresponding to the smaller value, which was copied to the output list, to the next element in that list.
- Once one of the lists has been exhausted, copy any remaining elements from the other list to the output list.
- Return the output list (this should be a new list containing all elements in sorted order from the two sorted input lists).



## Parallel Algorithm Overview

For this project we will take a data parallel approach to mergesort, in which we will partition the unsorted input list into $n$ *equally* sized sublists and then use $n$ processes to sort the sublists in parallel.  Once each sublist has been sorted we will use a parallel merge that uses our processes to pairwise merge the sorted sublists until only one list remains, which should contain the sorted version of the original input list.  This implmentation does not need to be in-place, that is you may return a sorted copy of the input data.

**Note:** For this portion of the project you may optionally break the partitioning and merging steps into two functions (similar to the method prescribed above in the serial section), or simply implement the parallel mergesort as a single function.  


### Parallel MergeSort

- Partition the input list into $n$ *equally* sized partitions, where $n$ is the number of processes we wish to use (passed as an argument). 
- Create $n$ processes (**not threads**) and use your implementation of the `merge_sort` function to process one of the partitions. That is, each process should handle only one of the $n$ sublists.  If you like, you may use `multiprocessing.Pool` and `Pool.map` in your implementation. 
- Once the above has completed you should have $n$ sorted sublists of the original input list.
- Perform a parallel merge on the $n$ sorted sublist.
- Return the resulting sorted list.


### Parallel Merge

- Given $n$ sorted sublists.
- Iterate until only a single sorted list remains:
    - Create (or reuse) $\frac{m}{2}$ processes (initially $m=n$, but $m$ should decrease each iteration) and use your implementation of the `merge` function to pairwise merge the sorted sublists.  That is each process should use your `merge` function to merge two of the sorted sublists.
    - This should result in half as many sorted sublists.
    - Take care to handle the case when there are an odd number of sublists.  For instance remove the odd element from consideration during this iteration and add it back for the next iteration.
- Return the resulting sorted list.


# Instructions

In this project you will implement both serial and parallel versions of merge sort.  Due to a bug in multiprocessing some windows users may have issues using the regular notebook to complete their project.  As such windows users are encourage to use the 'parallel_sorting_winbug' notebook, in which the implementation will actually be written in 'parallel_winbug.py' and imported in the imports section below.

## Submitting Your Assignment

You will submit your completed assignment in two formats:

- Jupyter Notebook (.ipynb)
- HTML (.html)

##### Jupyter Notebook (.ipynb)
You may directly use this notebook to complete your assignment or you may use an external editor/IDE of your choice.  However, to submit your code please ensure that your code works in this notebook.  
  
##### HTML (.html)
To create an HTML file for your assignment simply select `File > Download as > HTML (.html)` from within the Jupyter Notebook.  
  
Both files should be uploaded to [Canvas](https://canvas.tamu.edu).

## Imports

In [2]:
import math
import random
import time
import multiprocessing as mp

# Specifies the method to be used for creating processes
import sys
if sys.platform.startswith("linux"):  # linux
    mp.set_start_method('fork', force=True)
elif sys.platform.startswith("darwin"): # mac
    mp.set_start_method('fork', force=True)
else: # Assume Windows
    mp.set_start_method('spawn', force=True) 

## MergeSort

The implementation of merge_sort below is a top down approach that recursively divides the input into sublists of size 1.  Following that the sublists are merged such that the resuliting larger sublist remains sorted.  This implementation also modifies the input in-place and assumes that the input is indexable, like a list.

## Solution (1)

Complete the `merge` function below.  

In [3]:
def merge(data_tup):
    """
    Merges two sorted input lists into a single sorted list.  
    The sorted list returned contains a copy of the elements.
    
    data_tup: A tuple containing two sorted lists that are to 
              be merged, e.g. a,b = data_tup.  
    Returns a single sorted list containing a copy of all the 
    elements found in the two lists contained in data_tup.
    """
    arrayLeft, arrayRight = data_tup
    mergedArray = []
    left_idx, right_idx = 0, 0
    while left_idx < len(arrayLeft) and right_idx < len(arrayRight):
        if arrayLeft[left_idx] <= arrayRight[right_idx]:
            mergedArray.append(arrayLeft[left_idx])
            left_idx += 1
        else:
            mergedArray.append(arrayRight[right_idx])
            right_idx += 1
        
    if left_idx < len(arrayLeft):
        mergedArray.extend(arrayLeft[left_idx:])
    if right_idx < len(arrayRight):
        mergedArray.extend(arrayRight[right_idx:])
    return mergedArray

## Solution (2)

Complete the `merge_sort` function below.  

In [4]:
def merge_sort(data):
    """
    Recursively sorts the passed data list in ascending order.
    
    data: A list containing values to be sorted.
    
    Returns a single sorted list containing a copy of all the 
    elements found in the input data.
    """
    if len(data) <= 1:
        return data
    
    middle = len(data) // 2
    left = data[:middle]
    right = data[middle:]

    left = merge_sort(left)
    right = merge_sort(right)
    
    data_tup = left, right
    
    return merge(data_tup)


## Test Code (Sanity Check)

After completing the `merge` and `merge_sort` functions run the following code block to check if your functions produce a sorted list for a randomly generated list of `n` elements.  Even if your functions pass this test, bugs still may exist.  You should write your own test functions and verify your implementations.

In [5]:
def is_sorted(elements):
    """
    Returns true if elements are in sorted order
    """
    return elements == sorted(elements)

# Create some random test data
size = 100
data = random.sample(range(1000), size)
sorted_data = merge_sort(data)

if is_sorted(sorted_data):
    print('merge_sort passed sort test')
else:
    print('merge_sort failed sort test')
    print(sorted_data)


merge_sort passed sort test


## Solution (3)

Complete the `merge_sort_parallel` function below. 

In [6]:
from multiprocessing import Pool

def merge_sort_parallel(data, num_processes=None):
    """
    Data parallel implementation of mergesort.  Sorts the 
    passed data list (in ascending order) in parallel using 
    several processes.  Each process first sorts a portion 
    of the input data, then merges pairwise the sorted sublists
    until a single sorted list remains.
    
    data: A list containing values to be sorted.
    num_processes: The number of processes to be created and used
                   to partition and sort the data.  By default 
                   this value is mp.cpu_count() // 2 
    
    Returns a single sorted list containing a copy of all the 
    elements found in the input data.    
    """
    if num_processes is None:
        num_processes = mp.cpu_count()//2
    size_chunks = len(data)//num_processes
    chunk_list = [data[i:i+size_chunks] for i in range(0, len(data), size_chunks)]
    pool = Pool(processes = num_processes)
    sorted_sublists = pool.map(merge_sort, chunk_list)
    
    while len(sorted_sublists) > 1:
        chunk_list = [(sorted_sublists[i], sorted_sublists[i+1]) for i in range(0, len(sorted_sublists)-1, 2)]
        sorted_sublists = mpool.map(merge, chunk_list)
    
    return sorted_sublists[0]

## Test Code (Time Comparison)

After completing the `merge_sort` and `merge_sort_parallel` functions run the following code blocks to compare the runtime performance of your implementations.  Note that these code blocks simply time your code, it does not test or verify that the output of your functions is correct.


In [ ]:
print('This system contains', mp.cpu_count(), 'cores, or half this many if hyperthreading is supported')
print('This test used the default number of processes, which should be', mp.cpu_count() // 2)
size = 1000000
data = [random.randint(0, size) for _ in range(size)]

for sort in [merge_sort, merge_sort_parallel]:
    start = time.time()
    sorted_data = sort(data)
    duration = time.time() - start
    print(sort.__name__, duration, is_sorted(sorted_data))

This system contains 8 cores, or half this many if hyperthreading is supported
This test used the default number of processes, which should be 4
merge_sort 7.395313739776611 True


## Solution (4)


If the above had been implemented using Python Threads how would you expect the parallel speed up to be affected?  Should this be more or less than the process-based implementation above?  Why?  For this you should assume the same algorithm is used for both threads and processes.

In terms of speed Python Threads should speed up the parallel. And compared to the process-based implementation it should be taking less time. The reason behind is because in the case of thread it shares the same memory within threads, so it takes less memory that needs to copied unlike processing.